In [5]:
import random
from math import floor

In [31]:
class RANDU:
    def __init__(self, mul, mod,seed):
        self.mul = mul
        self.mod = mod
        self.currvalue = seed
        self.seed = seed
    def rand(self):
        self.currvalue = (self.currvalue*self.mul) % self.mod
        return self.currvalue
    def rewind(self):
        self.currvalue = self.seed

class Generator:
    def __init__(self, j, k, seed):
        self.j = j
        self.k = k
        self.g = RANDU(65539, 2**31, seed)
        self.currvalue = [self.g.rand() for _ in range(k)]
        self.initial = self.currvalue
        self.seed = seed #список начальных значений
        self.currpos = 0
    def rand(self):
        self.currvalue[self.currpos] = self.currvalue[self.currpos] ^ self.currvalue[(self.currpos-self.j) % self.k]
        val = self.currvalue[self.currpos]
        self.currpos = (self.currpos+1)%self.k
        return val
    def rewind(self):
        self.currvalue = self.initial
        self.currpos = 0



In [32]:
seed = 1
gen = Generator(24, 55, seed)

In [33]:
for _ in range(20):
    print(gen.rand())

662650498
889043850
1517105810
1099405258
1556661794
1532516010
1658779090
347529642
209626306
678846154
1528339154
797728906
1466507682
854895658
1882716050
1158399658
97497090
1156067338
1107447826
236237002


In [34]:
gen.rewind()

65539
393225
1769499
7077969
26542323
95552217
334432395
1146624417
1722371299
14608041
1766175739
1875647473
1800754131
366148473
1022489195
692115265
1392739779
2127401289
229749723
1559239569
